In [1]:
import spacy
from spacy.language import Language
import os
import docx2txt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pnair\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pnair\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nlp = spacy.load('en_core_web_lg',disable = ['ner', 'parser'])
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(nlp.pipe_names)

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']


In [3]:
def all_file_paths(master_directory):
    path_l = []
    for root, dirs, files in os.walk(master_directory):
        if files:
            for i in files:
                path = "{}\{}".format(root,i)
                if path == 'bbc_doc\desktop.ini':
                    continue
                path_l.append(path)
                
    
    return(path_l)

all_path = all_file_paths('bbc_doc')

print(len(all_path))
for i in all_path:
    print(i)

25
bbc_doc\business\001.docx
bbc_doc\business\002.docx
bbc_doc\business\003.docx
bbc_doc\business\004.docx
bbc_doc\business\005.docx
bbc_doc\entertainment\001.docx
bbc_doc\entertainment\002.docx
bbc_doc\entertainment\003.docx
bbc_doc\entertainment\004.docx
bbc_doc\entertainment\005.docx
bbc_doc\politics\001.docx
bbc_doc\politics\002.docx
bbc_doc\politics\003.docx
bbc_doc\politics\004.docx
bbc_doc\politics\005.docx
bbc_doc\sports\001.docx
bbc_doc\sports\002.docx
bbc_doc\sports\003.docx
bbc_doc\sports\004.docx
bbc_doc\sports\005.docx
bbc_doc\technology\001.docx
bbc_doc\technology\002.docx
bbc_doc\technology\003.docx
bbc_doc\technology\004.docx
bbc_doc\technology\005.docx


In [4]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    return text


def findline(word,doc_list): #Added doc_list as var
    line_num = []
    line_list = []
    for i in range(len(doc_list)):
        if word in doc_list[i]:
            line_num.append(i+1)
            line_list.append(doc_list[i])

    return line_num, line_list

def word_occurences(text, tokens_str, query):
    
#     wordfreq = []
#     for w in tokens_str:
#         wordfreq.append(tokens_str.count(w))


#     for i in list(zip(tokens_str, wordfreq)):
#         if query == i[0]:
#             word_occurences = (i[0], i[1])
#             break

#     line = 0
#     word_not_found = []

#     for word in text:
#         if word == '\n':
#             line += 1

    doc_list = text.lower().splitlines()
    doc_list = [i for i in doc_list if i]
    
    return doc_list

In [5]:
def sim_word(path,query,score):
 
#FI
    text = docx2txt.process(path)
    string_txt = str(text)

    tokens = nlp(text)
    lemma_list = []
    for token in tokens:
        if token.is_stop is False:
            token_preprocessed = preprocessor(token.text)
            if token_preprocessed != '':
                 lemma_list.append(nlp(token_preprocessed))
   

    tokens_str = text.lower()
    tokens_str = word_tokenize(tokens_str)
    tokens_str = [i for i in tokens_str if not i in stop_words]    
#FI    
    
    query = query
    
    l_q = len(nlp(query))
    
    if l_q >= 2:
        
        bi_g = []
        c=0
        while c < len(lemma_list)-1:

            bi_g.append(nlp(lemma_list[c].text+" "+lemma_list[c+1].text))
            c = c+1


        lemma_list = lemma_list+bi_g
    
#PN 
    doc_list = word_occurences(text, lemma_list, query)
    
    string_list = []
    paragraph_list = []
    paragraph_matrix_list = []
    
    string_list.append(string_txt)
    
    string_list = string_list[0].splitlines()
    
    for word in string_list:
        if word != '':
            paragraph_list.append(word)
    
    #print(paragraph_list)
#PN


#FI & PN
    key = nlp(query)
    
    sim_score = []
    words = []
    word_count = []
    #paragraph_word_set = set()
   # paragraph_sentences = []

    pg_list = []
    line_list_words = []

    for i in lemma_list:
        s = key.similarity(i)

        if s > score:
            words.append(i.text)
                
            # PN
            sim_score.append(s)

            word_found_list = []
            word_found_list.append(str(i.text))
            word_found_lines, line_list = findline(word_found_list[0],doc_list)
            #print(line_list)
            l_line_list = []
            l_line_list.append(tuple(line_list))
            paragraph_lines = list(line_list)
            #paragraph_lines = [ele for ele in paragraph_lines if ele != []]
            word_count.append(len(word_found_lines))
            
            pg_list.append(tuple(word_found_lines))
            line_list_words.append(tuple(line_list))
    
    matrix = pd.DataFrame({'Words': words, 'Similarity Score': sim_score,'Paragraph Numbers': pg_list, 'Count': word_count, 'Paragraphs': line_list_words, 'Path': path}, index = words)

    #matrix = pd.DataFrame({'Similarity Score': sim_score,'Paragraph Numbers': pg_list, 'Count': word_count}, index = words)
    matrix = matrix.drop_duplicates()

    matrix =  matrix.sort_values(by=['Similarity Score'],ascending=False)
    #print("For document: {}, path: {}, keyword: {}, Similarity Score Threshold: {}".format(os.path.basename(path),path,query,score))
    #print(matrix)
    return matrix
   # print('\n')

#FI & PN

In [6]:
all_path[0] == 'bbc_doc\\business\\001.docx'

True

In [7]:
all_path.index('bbc_doc\\business\\001.docx')

0

In [8]:
output = sim_word(all_path[0],"profit",0.6)
output

,Words,Similarity Score,Paragraph Numbers,Count,Paragraphs,Path
profit,profit,1.000000,"(1, 2, 3, 4, 5)",5,"(ad sales boost time warner profit, quarterly ...",bbc_doc\business\001.docx
profits,profits,0.913009,"(2, 3, 4, 5)",4,(quarterly profits at us media giant timewarne...,bbc_doc\business\001.docx
revenue,revenue,0.791138,"(4, 5, 6)",3,(time warner said on friday that it now owns 8...,bbc_doc\business\001.docx
earnings,earnings,0.762458,"(5,)",1,(time warner's fourth quarter profits were sli...,bbc_doc\business\001.docx
revenues,revenues,0.735938,"(4, 5)",2,(time warner said on friday that it now owns 8...,bbc_doc\business\001.docx
sales,sales,0.627998,"(1, 3)",2,"(ad sales boost time warner profit, the firm, ...",bbc_doc\business\001.docx
gains,gains,0.620257,"(3,)",1,"(the firm, which is now one of the biggest inv...",bbc_doc\business\001.docx
pay,pay,0.602320,"(6,)",1,(timewarner is to restate its accounts as part...,bbc_doc\business\001.docx


In [9]:
word_df = []
final_df = []

In [10]:
# Check unigram,bigram, web app interface, check time complexity with 300 pages dataset, similar to google search from 1-100(access index of dataframe list: click 1 will trigger i-1 index of list)
path = 'bbc_doc\business\001.docx'
for i in all_path[:]:
    output = sim_word(i,"profit",0.6)
    word_df.append(output)

In [11]:
word_df = [df for df in word_df if not df.empty] # removing dataframes that are empty
word_df

[             Words  Similarity Score Paragraph Numbers  Count  \
 profit      profit          1.000000   (1, 2, 3, 4, 5)      5   
 profits    profits          0.913009      (2, 3, 4, 5)      4   
 revenue    revenue          0.791138         (4, 5, 6)      3   
 earnings  earnings          0.762458              (5,)      1   
 revenues  revenues          0.735938            (4, 5)      2   
 sales        sales          0.627998            (1, 3)      2   
 gains        gains          0.620257              (3,)      1   
 pay            pay          0.602320              (6,)      1   
 
                                                  Paragraphs  \
 profit    (ad sales boost time warner profit, quarterly ...   
 profits   (quarterly profits at us media giant timewarne...   
 revenue   (time warner said on friday that it now owns 8...   
 earnings  (time warner's fourth quarter profits were sli...   
 revenues  (time warner said on friday that it now owns 8...   
 sales     (ad sales

In [12]:
# dropping rows with Count having a value of 0
for df in word_df:
    df = df[df.Count != 0]
    final_df.append(df)

In [13]:
df = pd.concat(final_df)
df

,Words,Similarity Score,Paragraph Numbers,Count,Paragraphs,Path
profit,profit,1.000000,"(1, 2, 3, 4, 5)",5,"(ad sales boost time warner profit, quarterly ...",bbc_doc\business\001.docx
profits,profits,0.913009,"(2, 3, 4, 5)",4,(quarterly profits at us media giant timewarne...,bbc_doc\business\001.docx
revenue,revenue,0.791138,"(4, 5, 6)",3,(time warner said on friday that it now owns 8...,bbc_doc\business\001.docx
earnings,earnings,0.762458,"(5,)",1,(time warner's fourth quarter profits were sli...,bbc_doc\business\001.docx
revenues,revenues,0.735938,"(4, 5)",2,(time warner said on friday that it now owns 8...,bbc_doc\business\001.docx
sales,sales,0.627998,"(1, 3)",2,"(ad sales boost time warner profit, the firm, ...",bbc_doc\business\001.docx
gains,gains,0.620257,"(3,)",1,"(the firm, which is now one of the biggest inv...",bbc_doc\business\001.docx
pay,pay,0.602320,"(6,)",1,(timewarner is to restate its accounts as part...,bbc_doc\business\001.docx
dollars,dollars,0.625726,"(4,)",1,(worries about the deficit concerns about chin...,bbc_doc\business\002.docx
gains,gains,0.620257,"(1, 2, 3)",3,"(dollar gains on greenspan speech, the dollar ...",bbc_doc\business\002.docx
